In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.1 MB/s eta 0:00:00


In [1]:
import pandas as pd

df = pd.read_json("/content/drive/MyDrive/NLP/Project/datasets.json")

In [2]:
import random
def make_trait(row):
    tweets_selected = random.sample(row["tweets"], 15 if len(row["tweets"]) > 15 else len(row["tweets"]))
    row["tweets_text"] = " ".join(tweets_selected)
    row["tweets"] = tweets_selected
    return row
df = df.apply(make_trait, axis=1)

In [16]:
import openai

openai.api_key = 'my api key'
counter = 0

def zero_shot_classifier(row):
  global counter
  counter += 1
  if counter % 5 == 0:
    print(f"on user {counter}")
  tweets = row["tweets_text"]
  prompt = f"You are picking the labels for a zero shot classification model on user tweets, domain is MBTI personality types, Generate a label from 16 MBTI types."
  prompt += f"tweet is : `{tweets}`"
  chat = openai.Completion.create(
      engine="davinci",
      prompt=prompt,
      temperature=0.3,
      max_tokens=16,
      n=1,
      stop=None
    )
  reply = chat.choices[0].text.strip().strip("\n").replace("label", "").replace("tweet", "").replace(":", "").replace("Output", "").replace("output", "").replace("Tweet", "").replace("Answer", "")
  row["GPT_result"] = reply
  # print("chatgpt : ", reply)
  return row


In [17]:
df = df.iloc[:120]

df_out = df.apply(zero_shot_classifier, axis=1)

RateLimitError: ignored

In [ ]:
def evaluate_GPT(row):
  row["equal_0"] = 1 if row["mbti_result"][0] == row["GPT_result"][0] else 0
  row["equal_1"] = 1 if row["mbti_result"][1] == row["GPT_result"][1] else 0
  row["equal_2"] = 1 if row["mbti_result"][2] == row["GPT_result"][2] else 0
  row["equal_3"] = 1 if row["mbti_result"][3] == row["GPT_result"][3] else 0
df_out = df_out.apply(evaluate_GPT, axis=1)
acc = {}

for i in range(4):
  acc[f"trait_{i}"] = round(df[f"equal_{i}"].sum() / len(df), 2)